In [272]:
# Import the libraries
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import json
import folium
from folium.vector_layers import Circle

In [2]:
# Access to the database in MongoDB
client=MongoClient()
db=client.get_database("companies")

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
load_dotenv()

True

In [5]:
# Get the tokens that were saved in the .env
gp_token=os.getenv("GOOGLE_P_TOKEN")
fq_client_id=os.getenv("FQ_CLIENT_ID")
fq_client_secret=os.getenv("FQ_CLIENT_SECRET")

# First screening with MongoDB operator

- I used $near operator to select the companies that are in Silicon Valley (California), because the big tech companies are stablished there.
- I also searched for companies that were near the airport, I looked for companies near the San Francisco airport but as there was only one, I then looked for companies near the other big airport in that area which is the San José airport.

In [70]:
# Create a new database named companies_unwinded, so that I do not 
# have several offices for the same company

res = db.companies.aggregate([
                      {"$unwind":"$offices" },
                      {"$match":{"offices.latitude":{"$ne":None}}},
                      {"$match":{"offices.longitude":{"$ne":None}}},
                      {"$project":{"_id":0}}])

db.companies_unwinded.insert_many(res)

offices = db.companies_unwinded

res = offices.find({},{"offices":1})


for c in res:
    filt = {"_id":c["_id"]}
    coord = {
              "type": "Point",
              "coordinates": [c["offices"]["longitude"], c["offices"]["latitude"]]
            }
    update = {"$set":{"coord":coord}}
    offices.update_one(filt,update)


In [71]:
type(res)

pymongo.cursor.Cursor

In [66]:
# Look for the companies that are near San Francisco airport

point = {"type":"Point",
         "coordinates":[-122.390091, 37.615404]}

query ={
            "coord":{
                "$near":{
                    "$geometry":point,
                    "$maxDistance":2500,
                    #"$minDistance":
                }
            }
        }

res = db.companies_unwinded.find(query,{"offices":1})
list(res)

# There is only one so I look for the companies near San José airport

[{'_id': ObjectId('5fafa246e21df4eac0a71b53'),
  'offices': {'description': 'Corporate Headquarters',
   'address1': '1745 Adrian Rd. Suite 1',
   'address2': '',
   'zip_code': '94010',
   'city': 'Burlingame',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.598478,
   'longitude': -122.376572}}]

In [319]:
# Look for the companies near San José airport


point = {"type":"Point",
         "coordinates":[-121.928927, 37.363803]}

query ={
            "coord":{
                "$near":{
                    "$geometry":point,
                    "$maxDistance":2000,
                    #"$minDistance":
                }
            }
        }


res1 = db.companies_unwinded.find(query, {"offices":1})
list(res1)
# There are many companies that are at 1km or less from this airport

[{'_id': ObjectId('5fafa245e21df4eac0a707f2'),
  'offices': {'description': 'Finjan Holdings, Inc',
   'address1': '122 East 42nd Street',
   'address2': 'Suite 1512',
   'zip_code': '10168',
   'city': 'New York',
   'state_code': 'NY',
   'country_code': 'USA',
   'latitude': 37.37027,
   'longitude': -121.924053}},
 {'_id': ObjectId('5fafa246e21df4eac0a709d1'),
  'offices': {'description': 'Finjan Holdings, Inc',
   'address1': '122 East 42nd Street',
   'address2': 'Suite 1512',
   'zip_code': '10168',
   'city': 'New York',
   'state_code': 'NY',
   'country_code': 'USA',
   'latitude': 37.37027,
   'longitude': -121.924053}},
 {'_id': ObjectId('5fafa246e21df4eac0a71df5'),
  'offices': {'description': 'USA Headquarters',
   'address1': '2025 Gateway Place, Suite 400',
   'address2': '',
   'zip_code': '95110',
   'city': 'San Jose',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.37027,
   'longitude': -121.924053}},
 {'_id': ObjectId('5fb012d0f93b0b3e63f74b44')

I realized that there are some companies that are duplicated, so the total number of companies is reduced to 5.

# First company analysis

In [277]:
def api_function(ll, query, name):
    url = 'https://api.foursquare.com/v2/venues/explore'

    params = dict(
    client_id=fq_client_id,
    client_secret=fq_client_secret,
    v='20201015',
    ll=ll,
    query=query,
    limit=15
    )
    resp = requests.get(url=url, params=params)
    name = json.loads(resp.text)
    return name

In [321]:
# Coordinates of the first company
res1[10]["offices"]["city"]

InvalidOperation: cannot set options after executing query

In [278]:
# I selected the first company on the list above which is located in Sunnyvale, an area that is near to Google and Facebook offices
# I make a query to the Foursquare API to look for tech startups near this company.
api_function('37.3709977,-121.9235529', 'Tech Startup', 'tech_startups_sun1')

{'meta': {'code': 200, 'requestId': '5fb28b6f71cf8433225eebf5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'tech startup',
  'totalResults': 173,
  'suggestedBounds': {'ne': {'lat': 37.37629187071258,
    'lng': -121.91692836349999},
   'sw': {'lat': 37.3671136632978, 'lng': -121.9260063665}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51c0abff498e828260d4a5cc',
       'name': 'PayPal',
       'location': {'address': '2211 N 1st St',
        'lat': 37.375874679466456,
        'lng': -121.92306376202592,
        'labeled

In [157]:
def dist(lista):
    distances=[]
    for x in range(0,14):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["distance"]
        distances.append(result)
    return distances

dist(tech_startups_sun1)

[544, 171, 170, 229, 256, 276, 367, 368, 434, 446, 485, 596, 181, 608]

In [163]:
def lat(lista):
    latitude=[]
    for x in range(0,14):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["lat"]
        latitude.append(result)
    return latitude
lat(tech_startups_sun1)

[37.375874679466456,
 37.370972648019674,
 37.36947,
 37.36957044583222,
 37.3695173976203,
 37.369302000000005,
 37.37009996545508,
 37.371297,
 37.369042,
 37.36805224766053,
 37.36753085454393,
 37.368907,
 37.37114012,
 37.36766592556348]

In [164]:
def lng(lista):
    longitude=[]
    for x in range(0,14):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["location"]["lng"]
        longitude.append(result)
    return longitude

lng(tech_startups_sun1)

[-121.92306376202592,
 -121.92549178587224,
 -121.92351,
 -121.921677387159,
 -121.92133577652343,
 -121.92127099999999,
 -121.91955220035622,
 -121.9194084405899,
 -121.919304,
 -121.92011894731519,
 -121.92021896118176,
 -121.917341,
 -121.92559373,
 -121.9180963651028]

In [166]:
def name(lista):
    names=[]
    for x in range(0,14):
        result=lista["response"]["groups"][0]["items"][x]["venue"]["name"]
        names.append(result)
    return names

name(tech_startups_sun1)

['PayPal',
 'SourceN HQ',
 'FixStream',
 'River Meadow Software',
 'Robin Systems',
 'PlayPhone, Inc.',
 'skytree',
 'Ocarina Networks',
 'SVACPA',
 'Gaia Online',
 'Ensighten',
 'GEO Seminconductor',
 'DataTech Labs Data Recovery',
 'Nutanix HQ']

In [55]:
# I search for the nearest tech startups to the offices of the 
# company I selected
dist_tech=tech_startups_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_tech=tech_startups_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_tech=tech_startups_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_tech=tech_startups_sun1["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: PayPal, company coordinates: 37.375874679466456,-121.92306376202592, distance to the offices: 544


The nearest company is PayPal which is at 544m from the offices.

In [279]:
# I make a query to the Foursquare API to look for design studios near this company.

api_function('37.3709977,-121.9235529', 'Design Studio', 'design_studio_sun1')


{'meta': {'code': 200, 'requestId': '5fb28bbe4467db32a3211e43'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'design studio',
  'totalResults': 55,
  'suggestedBounds': {'ne': {'lat': 37.38488118593506, 'lng': -121.90373605},
   'sw': {'lat': 37.36479886733642, 'lng': -121.94608495000001}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '555cc5ae498e23396776d0b1',
       'name': 'Kiskon Technologies',
       'location': {'address': '2076 Gateway Pl # 500',
        'lat': 37.37334604119023,
        'lng': -121.92193508148193,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.37334604119023,
          'lng': -121.9

In [173]:
print(dist(design_studio_sun1))
print("----------------------------------------------")
print(lat(design_studio_sun1))
print("----------------------------------------------")
print(lng(design_studio_sun1))
print("----------------------------------------------")
print(name(design_studio_sun1))

[298, 785, 582, 1274, 1291, 1345, 1351, 1359, 1465, 1490, 1587, 1614, 1765, 1401]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.37102127075195, 37.378426714690484, 37.3707160949707, 37.374528, 37.373359, 37.374013, 37.379857, 37.373688, 37.373843, 37.383968353271484, 37.3657117]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.90914154052734, -121.91234536963067, -121.9083480834961, -121.908934, -121.90847, -121.907427, -121.910925, -121.905933, -121.905661, -121.91206359863281, -121.9091752]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Screened Magic', 'Home Expo &  Design', 'Woodenbridge', 'Summit Cabinet', 'The Ceramic Corporation', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Zeus Tile', 'Emser Tile&Natural Stone', 'Signarama San Jose']


In [59]:
dist_design=design_studio_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_design=design_studio_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_design=design_studio_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_design=design_studio_sun1["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

dist_design=design_studio_sun1["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_design=design_studio_sun1["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_design=design_studio_sun1["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_design=design_studio_sun1["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Kiskon Technologies, company coordinates: 37.37334604119023,-121.92193508148193, distance to the offices: 298
company name: Konica Minolta Business Solutions, company coordinates: 37.3691219,-121.917411, distance to the offices: 582


The nearest studio is at 298m and there is another one at 582m from the office.

In [280]:
# I make a query to the Foursquare API to look for schools near this company.

api_function('37.3709977,-121.9235529', 'Elementary School', 'elementary_sch_sun1')


{'meta': {'code': 200, 'requestId': '5fb28bfaa5d23e3a411d4e93'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'elementary school',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 37.409873762716295,
    'lng': -121.87983937403224},
   'sw': {'lat': 37.32654227934447, 'lng': -121.96274533744061}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c925989b641236a58358979',
       'name': 'Bachrodt Elementary School',
       'location': {'address': '102 Sonora Ave',
        'lat': 37.36330687882652,
        'lng': -121.9127954630632,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.36330687882652,
         

In [182]:
print(dist(elementary_sch_sun1))
print("----------------------------------------------")
print(lat(elementary_sch_sun1))
print("----------------------------------------------")
print(lng(elementary_sch_sun1))
print("----------------------------------------------")
print(name(elementary_sch_sun1))

[1280, 2768, 2901, 3224, 3362, 3460, 3481, 3140, 3658, 3856, 3967, 3675, 4504, 4514]
----------------------------------------------
[37.36330687882652, 37.388926007147695, 37.38955552112065, 37.39309384415278, 37.358551025390625, 37.343393216725616, 37.35738251403415, 37.347336, 37.348379, 37.350991, 37.38721347682061, 37.398261959325076, 37.40608596801758, 37.333647875243265]
----------------------------------------------
[-121.9127954630632, -121.90186142921448, -121.94658055857437, -121.94713050142707, -121.95817565917969, -121.94153776982016, -121.9589768845584, -121.942885, -121.893555, -121.88796799999999, -121.88360782691444, -121.94699718220039, -121.898193359375, -121.94342527769899]
----------------------------------------------
['Bachrodt Elementary School', 'Orchard Elementary School', 'Montague Elementary School', 'Don Callejon School', 'Jano Hindi Bee', 'Washington Open Elementary', 'Scott Lane Elementary School', "St Clare's Catholic School", 'Suzume No Gakko', 'Grant El

In [57]:
dist_school=elementary_sch_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_school=elementary_sch_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_school=elementary_sch_sun1["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_school=elementary_sch_sun1["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Bachrodt Elementary School, company coordinates: 37.36330687882652,-121.9127954630632, distance to the offices: 1280


The nearest school is at 1.2km

In [281]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

api_function('37.3709977,-121.9235529', 'Nightlife Spot', 'nightlife_spot_sun1')


{'meta': {'code': 200, 'requestId': '5fb28c255111134f6b2bc0cf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'nightlife spot',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 37.376633580868805,
    'lng': -121.90005185238185},
   'sw': {'lat': 37.357700801755165, 'lng': -121.95620128149342}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d97ef6a97d06ea85146290b',
       'name': 'The Doghouse Sports Lounge',
       'location': {'address': '1336 N 1st St',
        'crossStreet': 'Rosemary',
        'lat': 37.3608659956

In [179]:
print(dist(nightlife_spot_sun1))
print("----------------------------------------------")
print(lat(nightlife_spot_sun1))
print("----------------------------------------------")
print(lng(nightlife_spot_sun1))
print("----------------------------------------------")
print(name(nightlife_spot_sun1))

[298, 785, 582, 1274, 1291, 1345, 1351, 1359, 1465, 1490, 1587, 1614, 1765, 1401]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.37102127075195, 37.378426714690484, 37.3707160949707, 37.374528, 37.373359, 37.374013, 37.379857, 37.373688, 37.373843, 37.383968353271484, 37.3657117]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.90914154052734, -121.91234536963067, -121.9083480834961, -121.908934, -121.90847, -121.907427, -121.910925, -121.905933, -121.905661, -121.91206359863281, -121.9091752]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Screened Magic', 'Home Expo &  Design', 'Woodenbridge', 'Summit Cabinet', 'The Ceramic Corporation', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Zeus Tile', 'Emser Tile&Natural Stone', 'Signarama San Jose']


In [69]:
dist_party=nightlife_spot_sun1["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sun1["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sun1["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=nightlife_spot_sun1["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=nightlife_spot_sun1["response"]["groups"][0]["items"][9]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sun1["response"]["groups"][0]["items"][9]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sun1["response"]["groups"][0]["items"][9]["venue"]["location"]["lng"]
name_party=nightlife_spot_sun1["response"]["groups"][0]["items"][9]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")


company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 123
company name: 2050 Lobby Lounge & Bar, company coordinates: 37.371769553739405,-121.92226542378253, distance to the offices: 142


There are two nighlife sports near, one at 123m and other at 142m

In [282]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

api_function('37.3709977,-121.9235529', 'Vegan Restaurant', 'veg_rest_sun1')


{'meta': {'code': 200, 'requestId': '5fb28c4269bc8506c4354457'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'vegan restaurant',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 37.42560895276847,
    'lng': -121.88480138993586},
   'sw': {'lat': 37.32140595492598, 'lng': -121.9817248113471}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '507e209ae4b0959d8bc30ecc',
       'name': "Ike's Sandwiches",
       'location': {'address': '2235 The Alameda',
        'crossStreet': 'at Chapman Ct',
        'lat': 37.3446038364260

In [183]:
print(dist(veg_rest_sun1))
print("----------------------------------------------")
print(lat(veg_rest_sun1))
print("----------------------------------------------")
print(lng(veg_rest_sun1))
print("----------------------------------------------")
print(name(veg_rest_sun1))

[3051, 3394, 5320, 2120, 2619, 2436, 3399, 3762, 3955, 4061, 4145, 4858, 5206, 5589]
----------------------------------------------
[37.34460383642609, 37.343718446032305, 37.32614245482791, 37.385706, 37.35659, 37.3607916, 37.34049606323242, 37.33930772, 37.338702000000005, 37.404856, 37.396342, 37.41086084497713, 37.35198682571322, 37.42087245286653]
----------------------------------------------
[-121.93286117841694, -121.9406861466985, -121.94432964084655, -121.908328, -121.94696, -121.8991962, -121.92169952392578, -121.93835143, -121.942205, -121.906444, -121.88920700000001, -121.94593038741806, -121.97731920128297, -121.91625220186718]
----------------------------------------------
["Ike's Sandwiches", "deedee's", 'Loving Hut', 'Vegefarm', 'The Happy Hooligans', "Brother Baby's BBQ", 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie', 'Tasty Vegetarian Cuisine', 'SJ Vegetarian', "Sri's Kitchen", 'Peacock Indian Cuisine', 'Loving Hut']


In [74]:
dist_veg=veg_rest_sun1["response"]["groups"][0]["items"][3]["venue"]["location"]["distance"]
lat_veg=veg_rest_sun1["response"]["groups"][0]["items"][3]["venue"]["location"]["lat"]
lng_veg=veg_rest_sun1["response"]["groups"][0]["items"][3]["venue"]["location"]["lng"]
name_veg=veg_rest_sun1["response"]["groups"][0]["items"][3]["venue"]["name"]

print(f"company name: {name_veg}, company coordinates: {lat_veg},{lng_veg}, distance to the offices: {dist_veg}")


company name: Brother Baby's BBQ, company coordinates: 37.3607916,-121.8991962, distance to the offices: 2436


The nearest restaurant is at 2.4km from the office.

In [283]:
# I make a query to the Foursquare API to look for basketball courts near this company.

api_function('37.3709977,-121.9235529', 'Basketball Court', 'basketball_court_sun1')


{'meta': {'code': 200, 'requestId': '5fb28c5fa0021654cac3975a'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'basketball court',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 37.420003872877636,
    'lng': -121.87698936462401},
   'sw': {'lat': 37.32647170912039, 'lng': -121.97387886047365}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a2f2f67f964a520b3981fe3',
       'name': 'Central YMCA',
       'location': {'address': '1717 The Alameda',
        'crossStreet': 'at Naglee Ave.',
        'lat': 37.33779578622328,
        'lng': -121.91989631992244,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [186]:
print(dist(basketball_court_sun1))
print("----------------------------------------------")
print(lat(basketball_court_sun1))
print("----------------------------------------------")
print(lng(basketball_court_sun1))
print("----------------------------------------------")
print(name(basketball_court_sun1))

[3710, 668, 1036, 1071, 1119, 2757, 3022, 3556, 3970, 4091, 4679, 4265, 4483, 4794]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.37117385864258, 37.364402, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.3753662109375, 37.40387815016547, 37.33270914406301, 37.330723171109355, 37.3980712890625]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.91143798828125, -121.913997, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.96947479248047, -121.9565124356151, -121.9253028797591, -121.92286373936807, -121.88139343261719]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Basketball Gym', 'Skyport!', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Podium Golden State Warriors Basketball Court', 'Lick Mill Park'

In [86]:
dist_basket=basketball_court_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basketball_court_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basketball_court_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basketball_court_sun1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 668


The nearest one is at 668m

In [284]:
# I make a query to the Foursquare API to look for Starbucks near this company.

api_function('37.3709977,-121.9235529', 'starbucks', 'starbucks_sun1')


{'meta': {'code': 200, 'requestId': '5fb28c94164c641f7b27db6a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'starbucks',
  'totalResults': 50,
  'suggestedBounds': {'ne': {'lat': 37.4100811785, 'lng': -121.89204880000001},
   'sw': {'lat': 37.3305692515, 'lng': -121.94412720000001}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a6b4b07f964a5205cce1fe3',
       'name': 'Starbucks',
       'location': {'address': '1751 N 1st St',
        'lat': 37.37092547570862,
        'lng': -121.9

In [189]:
print(dist(starbucks_sun1))
print("----------------------------------------------")
print(lat(starbucks_sun1))
print("----------------------------------------------")
print(lng(starbucks_sun1))
print("----------------------------------------------")
print(name(starbucks_sun1))

[555, 110, 550, 1196, 340, 4098, 2273, 3077, 3139, 3739, 1647, 3455, 3024, 2915]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.33418343, 37.35060173, 37.345295, 37.395157, 37.40197092074824, 37.385382, 37.341431, 37.398163, 37.38324096]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9241052, -121.9221276, -121.936371, -121.90523999999999, -121.93991940591599, -121.927919, -121.911642, -121.923706, -121.894416]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [90]:
dist_starbucks=starbucks_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks_sun1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

dist_starbucks=starbucks_sun1["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sun1["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sun1["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_starbucks=starbucks_sun1["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")


company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 110
company name: Starbucks, company coordinates: 37.369186,-121.92045, distance to the offices: 340


There are two near Starbucks, one at 100m and other at 340m.

In [285]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

api_function('37.3709977,-121.9235529', 'Pet Service', 'pet_service_sun1')


{'meta': {'code': 200, 'requestId': '5fb28caea5421e17cc5c59a0'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'pet service',
  'totalResults': 33,
  'suggestedBounds': {'ne': {'lat': 37.40474107281321,
    'lng': -121.88153774475427},
   'sw': {'lat': 37.32485747092281, 'lng': -121.98113218854394}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '49f6928cf964a520506c1fe3',
       'name': 'PetSmart',
       'location': {'address': '607 Coleman Ave',
        'crossStreet': 'at W Taylor St',
        'lat': 37.339942,
        'lng': -121.907791,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.339942,
          'lng'

In [192]:
print(dist(pet_service_sun1))
print("----------------------------------------------")
print(lat(pet_service_sun1))
print("----------------------------------------------")
print(lng(pet_service_sun1))
print("----------------------------------------------")
print(name(pet_service_sun1))

[3727, 2702, 4849, 3694, 5100, 2682, 2732, 2839, 2843, 2774, 3377, 3379, 3543, 3424]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.379786, 37.34972481944317, 37.40111, 37.38021, 37.34043631759206, 37.363336300242906]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.953731, -121.90721511840819, -121.91887, -121.95995900000001, -121.91236495971678, -121.8860647649265]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Epic Dog Park', 'Wag Hotels', 'Primal Canine', 'San Jose Animal Hospital']


In [109]:
dist_dog=pet_service_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_dog=pet_service_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_dog=pet_service_sun1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_dog=pet_service_sun1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")


company name: Banfield Pet Hospital, company coordinates: 37.3813788,-121.8959331, distance to the offices: 2702


There is one at 2.7km

Conclusion for this company:
- 

-----------------------------

# Second company analysis

In [286]:
# I make a query to the Foursquare API to look for tech startups near this company.

api_function('37.37027,-121.924053', 'Tech Startup', 'tech_startups_sj1')


{'meta': {'code': 200, 'requestId': '5fb28cd735436f34a0ecd92f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'tech startup',
  'totalResults': 178,
  'suggestedBounds': {'ne': {'lat': 37.37629187071258,
    'lng': -121.91692836349999},
   'sw': {'lat': 37.3671136632978, 'lng': -121.9260063665}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51c0abff498e828260d4a5cc',
       'name': 'PayPal',
       'location': {'address': '2211 N 1st St',
        'lat': 37.375874679466456,
        'lng': -121.92306376202592,
        'labeled

In [195]:
print(dist(tech_startups_sj1))
print("----------------------------------------------")
print(lat(tech_startups_sj1))
print("----------------------------------------------")
print(lng(tech_startups_sj1))
print("----------------------------------------------")
print(name(tech_startups_sj1))

[630, 149, 101, 224, 254, 268, 398, 426, 426, 441, 456, 167, 601, 612]
----------------------------------------------
[37.375874679466456, 37.370972648019674, 37.36947, 37.36957044583222, 37.3695173976203, 37.369302000000005, 37.37009996545508, 37.36805224766053, 37.371297, 37.369042, 37.36753085454393, 37.37114012, 37.36766592556348, 37.368907]
----------------------------------------------
[-121.92306376202592, -121.92549178587224, -121.92351, -121.921677387159, -121.92133577652343, -121.92127099999999, -121.91955220035622, -121.92011894731519, -121.9194084405899, -121.919304, -121.92021896118176, -121.92559373, -121.9180963651028, -121.917341]
----------------------------------------------
['PayPal', 'SourceN HQ', 'FixStream', 'River Meadow Software', 'Robin Systems', 'PlayPhone, Inc.', 'skytree', 'Gaia Online', 'Ocarina Networks', 'SVACPA', 'Ensighten', 'DataTech Labs Data Recovery', 'Nutanix HQ', 'GEO Seminconductor']


In [151]:
dist_tech=tech_startups_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_tech=tech_startups_sj1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech_startups_sj1["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj1["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj1["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_tech=tech_startups_sj1["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech_startups_sj1["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj1["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj1["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_tech=tech_startups_sj1["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: SourceN HQ, company coordinates: 37.370972648019674,-121.92549178587224, distance to the offices: 149
company name: FixStream, company coordinates: 37.36947,-121.92351, distance to the offices: 101
company name: DataTech Labs Data Recovery, company coordinates: 37.37114012,-121.92559373, distance to the offices: 167


There are three tech startups near, all of them aroun 150m.

In [317]:
# I make a query to the Foursquare API to look for design studios near this company.

api_function('37.37027,-121.924053', 'Design Studio', 'design_studio_sj1')


{'meta': {'code': 200, 'requestId': '5fb2904d5111134f6b3b1270'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'design studio',
  'totalResults': 60,
  'suggestedBounds': {'ne': {'lat': 37.38488118593506, 'lng': -121.90373605},
   'sw': {'lat': 37.36479886733642, 'lng': -121.94608495000001}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '555cc5ae498e23396776d0b1',
       'name': 'Kiskon Technologies',
       'location': {'address': '2076 Gateway Pl # 500',
        'lat': 37.37334604119023,
        'lng': -121.92193508148193,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.37334604119023,
          'lng': -121.9

In [198]:
print(dist(design_studio_sj1))
print("----------------------------------------------")
print(lat(design_studio_sj1))
print("----------------------------------------------")
print(lng(design_studio_sj1))
print("----------------------------------------------")
print(name(design_studio_sj1))

[390, 785, 601, 1321, 1377, 1390, 1419, 1420, 1528, 1577, 1647, 1674, 1410, 1845]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.37102127075195, 37.378426714690484, 37.3707160949707, 37.374528, 37.373359, 37.374013, 37.379857, 37.373688, 37.373843, 37.3657117, 37.365846999999995]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.90914154052734, -121.91234536963067, -121.9083480834961, -121.908934, -121.90847, -121.907427, -121.910925, -121.905933, -121.905661, -121.9091752, -121.94416000000001]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Screened Magic', 'Home Expo &  Design', 'Woodenbridge', 'Summit Cabinet', 'The Ceramic Corporation', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Zeus Tile', 'Signarama San Jose', 'All Architectural Stone']


In [129]:
dist_design=design_studio_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_design=design_studio_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_design=design_studio_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_design=design_studio_sj1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Watry Design, company coordinates: 37.3672899,-121.916, distance to the offices: 785


There is one design studio near at 785m

In [316]:
# I make a query to the Foursquare API to look for schools near this company.

api_function('37.37027,-121.924053', 'Elementary School', 'elementary_sch_sj1')


{'meta': {'code': 200, 'requestId': '5fb2903b69bc8506c443e3d2'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'elementary school',
  'totalResults': 32,
  'suggestedBounds': {'ne': {'lat': 37.40165855358917,
    'lng': -121.87178531076532},
   'sw': {'lat': 37.3269334797791, 'lng': -121.96312886426284}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c925989b641236a58358979',
       'name': 'Bachrodt Elementary School',
       'location': {'address': '102 Sonora Ave',
        'lat': 37.36330687882652,
        'lng': -121.9127954630632,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.36330687882652,
          '

In [201]:
print(dist(elementary_sch_sj1))
print("----------------------------------------------")
print(lat(elementary_sch_sj1))
print("----------------------------------------------")
print(lng(elementary_sch_sj1))
print("----------------------------------------------")
print(name(elementary_sch_sj1))

[1262, 2857, 2929, 3259, 3288, 3368, 3406, 3048, 3635, 3847, 4044, 3718, 4422, 4449]
----------------------------------------------
[37.36330687882652, 37.388926007147695, 37.38955552112065, 37.39309384415278, 37.358551025390625, 37.343393216725616, 37.35738251403415, 37.347336, 37.348379, 37.350991, 37.38721347682061, 37.398261959325076, 37.333647875243265, 37.3303300740432]
----------------------------------------------
[-121.9127954630632, -121.90186142921448, -121.94658055857437, -121.94713050142707, -121.95817565917969, -121.94153776982016, -121.9589768845584, -121.942885, -121.893555, -121.88796799999999, -121.88360782691444, -121.94699718220039, -121.94342527769899, -121.92598071783651]
----------------------------------------------
['Bachrodt Elementary School', 'Orchard Elementary School', 'Montague Elementary School', 'Don Callejon School', 'Jano Hindi Bee', 'Washington Open Elementary', 'Scott Lane Elementary School', "St Clare's Catholic School", 'Suzume No Gakko', 'Grant E

In [132]:
dist_school=elementary_sch_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_school=elementary_sch_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_school=elementary_sch_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_school=elementary_sch_sj1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Orchard Elementary School, company coordinates: 37.388926007147695,-121.90186142921448, distance to the offices: 2857


The nearest school is at 2.8km

In [315]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

api_function('37.37027,-121.924053', 'Nightlife Spot', 'nightlife_spot_sj1')


{'meta': {'code': 200, 'requestId': '5fb2902ba614367fd94fbb7a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'nightlife spot',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 37.37651835021626,
    'lng': -121.90555460815912},
   'sw': {'lat': 37.36012064545862, 'lng': -121.95593924550403}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d97ef6a97d06ea85146290b',
       'name': 'The Doghouse Sports Lounge',
       'location': {'address': '1336 N 1st St',
        'crossStreet': 'Rosemary',
        'lat': 37.360865995674

In [204]:
print(dist(nightlife_spot_sj1))
print("----------------------------------------------")
print(lat(nightlife_spot_sj1))
print("----------------------------------------------")
print(lng(nightlife_spot_sj1))
print("----------------------------------------------")
print(name(nightlife_spot_sj1))

[1730, 2801, 398, 1603, 514, 206, 222, 460, 478, 482, 584, 613, 647, 718]
----------------------------------------------
[37.36086599567488, 37.36132088609424, 37.37124135626228, 37.36381793355914, 37.365813193961756, 37.37152348661807, 37.371532546100354, 37.36690004129253, 37.366398, 37.369526, 37.375498869419765, 37.375773, 37.37026696954015, 37.36545102537579]
----------------------------------------------
[-121.90847504592831, -121.95364903471561, -121.91972331072563, -121.90784481894752, -121.9256078829507, -121.92232810486176, -121.9220969824811, -121.92706413888456, -121.92641, -121.929421, -121.92348539829254, -121.923609, -121.93136993764425, -121.9186465104381]
----------------------------------------------
['The Doghouse Sports Lounge', 'Golden State Brewery', 'Zone 8 Sports Bar & Grill', '4th St Bowl Lounge', 'Santa Cruz Wine Bar', 'Doubletree Pool', 'Fortyeight at DoubleTree', 'shark cage', 'T Baggers', 'San Jose Beer Union', 'GSPS Wine Friday', 'Please Be Thirsty', "Chia

In [156]:
dist_party=nightlife_spot_sj1["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj1["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj1["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj1["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=nightlife_spot_sj1["response"]["groups"][0]["items"][9]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj1["response"]["groups"][0]["items"][9]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj1["response"]["groups"][0]["items"][9]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj1["response"]["groups"][0]["items"][9]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=nightlife_spot_sj1["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj1["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj1["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj1["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

company name: Zone 8 Sports Bar & Grill, company coordinates: 37.37124135626228,-121.91972331072563, distance to the offices: 398
company name: 2050 Lobby Lounge & Bar, company coordinates: 37.371769553739405,-121.92226542378253, distance to the offices: 229
company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 206


There are three noghlife spots near around 200m and 300m.

In [314]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

api_function('37.37027,-121.924053', 'Vegan Restaurant', 'veg_rest_sj1')


{'meta': {'code': 200, 'requestId': '5fb29017f2dca11a8957a167'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'vegan restaurant',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 37.408791677258606,
    'lng': -121.87578680608625},
   'sw': {'lat': 37.322206777569306, 'lng': -121.9503491997102}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '507e209ae4b0959d8bc30ecc',
       'name': "Ike's Sandwiches",
       'location': {'address': '2235 The Alameda',
        'crossStreet': 'at Chapman Ct',
        'lat': 37.34460383642

In [207]:
print(dist(veg_rest_sj1))
print("----------------------------------------------")
print(lat(veg_rest_sj1))
print("----------------------------------------------")
print(lng(veg_rest_sj1))
print("----------------------------------------------")
print(name(veg_rest_sj1))

[2961, 4902, 4818, 5106, 3301, 5001, 5562, 5229, 2210, 2535, 2439, 3320, 3671, 3863]
----------------------------------------------
[37.34460383642609, 37.33547578302974, 37.336852209398536, 37.341433051588915, 37.343718446032305, 37.33993504196706, 37.330296164668816, 37.32614245482791, 37.385706, 37.35659, 37.3607916, 37.34049606323242, 37.33930772, 37.338702000000005]
----------------------------------------------
[-121.93286117841694, -121.89009385118939, -121.88943603809092, -121.87917600579642, -121.9406861466985, -121.88235410455921, -121.88632717974279, -121.94432964084655, -121.908328, -121.94696, -121.8991962, -121.92169952392578, -121.93835143, -121.942205]
----------------------------------------------
["Ike's Sandwiches", 'Good Karma', "Ike's Sandwiches", 'Vegetarian House', "deedee's", 'Tofoo Com Chay', 'Vitamina Juices & Blends', 'Loving Hut', 'Vegefarm', 'The Happy Hooligans', "Brother Baby's BBQ", 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie']


In [158]:
dist_vegan=veg_rest_sj1["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_vegan=veg_rest_sj1["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_vegan=veg_rest_sj1["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_vegan=veg_rest_sj1["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_vegan}, company coordinates: {lat_vegan},{lng_vegan}, distance to the offices: {dist_vegan}")

company name: Brother Baby's BBQ, company coordinates: 37.3607916,-121.8991962, distance to the offices: 2439


The nearest vegan restaurant is at 1.4km

In [313]:
# I make a query to the Foursquare API to look for basketball courts near this company.

api_function('37.37027,-121.924053', 'Basketball Court', 'basketball_court_sj1')


{'meta': {'code': 200, 'requestId': '5fb29005526d023dff4283c9'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'basketball court',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 37.420003872877636,
    'lng': -121.87698936462401},
   'sw': {'lat': 37.32647170912039, 'lng': -121.97387886047365}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a2f2f67f964a520b3981fe3',
       'name': 'Central YMCA',
       'location': {'address': '1717 The Alameda',
        'crossStreet': 'at Naglee Ave.',
        'lat': 37.33779578622328,
        'lng': -121.91989631992244,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [210]:
print(dist(basketball_court_sj1))
print("----------------------------------------------")
print(lat(basketball_court_sj1))
print("----------------------------------------------")
print(lng(basketball_court_sj1))
print("----------------------------------------------")
print(name(basketball_court_sj1))

[3633, 754, 1013, 1103, 1120, 2783, 2999, 3607, 3933, 4058, 4182, 4715, 4403, 4880]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.364402, 37.37117385864258, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.3753662109375, 37.33270914406301, 37.40387815016547, 37.330723171109355, 37.3980712890625]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.913997, -121.91143798828125, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.96947479248047, -121.9253028797591, -121.9565124356151, -121.92286373936807, -121.88139343261719]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Skyport!', 'Basketball Gym', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Podium Golden State Warriors Basketball Court', 'Hoover Gym', 'L

In [142]:
dist_basket=basketball_court_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basketball_court_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basketball_court_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basketball_court_sj1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 754


The nearest basketball court is at 754m

In [312]:
# I make a query to the Foursquare API to look for Starbucks near this company.

api_function('37.37027,-121.924053', 'starbucks', 'starbucks_sj1')


{'meta': {'code': 200, 'requestId': '5fb28ff8a5d23e3a412c0041'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'starbucks',
  'totalResults': 50,
  'suggestedBounds': {'ne': {'lat': 37.405360295285654,
    'lng': -121.8921408297042},
   'sw': {'lat': 37.3307940554626, 'lng': -121.94219457621179}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a6b4b07f964a5205cce1fe3',
       'name': 'Starbucks',
       'location': {'address': '1751 N 1st St',
        'lat': 37.37092547570862,
        'l

In [213]:
print(dist(starbucks_sj1))
print("----------------------------------------------")
print(lat(starbucks_sj1))
print("----------------------------------------------")
print(lng(starbucks_sj1))
print("----------------------------------------------")
print(name(starbucks_sj1))

[604, 198, 488, 1104, 340, 4017, 2196, 2986, 3392, 3231, 3797, 1716, 3105, 607]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.33418343, 37.35060173, 37.345295, 37.341431, 37.395157, 37.40197092074824, 37.385382, 37.398163, 37.36482957582107]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9241052, -121.9221276, -121.936371, -121.911642, -121.90523999999999, -121.93991940591599, -121.927919, -121.923706, -121.92356576168726]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [161]:
dist_starbucks=starbucks_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sj1["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks_sj1["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

dist_starbucks=starbucks_sj1["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sj1["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sj1["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_starbucks=starbucks_sj1["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 198
company name: Starbucks, company coordinates: 37.369186,-121.92045, distance to the offices: 340


There are two Starbucks near

In [311]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

api_function('37.37027,-121.924053', 'Pet Service', 'pet_service_sj1')


{'meta': {'code': 200, 'requestId': '5fb28fe812d8516a981358c4'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'pet service',
  'totalResults': 35,
  'suggestedBounds': {'ne': {'lat': 37.40474107281321,
    'lng': -121.8918994965814},
   'sw': {'lat': 37.32485747092281, 'lng': -121.98063877179027}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '49f6928cf964a520506c1fe3',
       'name': 'PetSmart',
       'location': {'address': '607 Coleman Ave',
        'crossStreet': 'at W Taylor St',
        'lat': 37.339942,
        'lng': -121.907791,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.339942,
          'lng':

In [217]:
print(dist(pet_service_sj1))
print("----------------------------------------------")
print(lat(pet_service_sj1))
print("----------------------------------------------")
print(lng(pet_service_sj1))
print("----------------------------------------------")
print(name(pet_service_sj1))

[3669, 2777, 4780, 3636, 5028, 2613, 2664, 2827, 2831, 2729, 3363, 3463, 3478, 3592]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.379786, 37.34972481944317, 37.38021, 37.40111, 37.34043631759206, 37.343559452656756]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.953731, -121.90721511840819, -121.95995900000001, -121.91887, -121.91236495971678, -121.94683830386161]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Wag Hotels', 'Epic Dog Park', 'Primal Canine', "Juan's Groom Room"]


In [162]:
dist_dog=pet_service_sj1["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_dog=pet_service_sj1["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_dog=pet_service_sj1["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_dog=pet_service_sj1["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")

company name: Ohana Kennels, company coordinates: 37.36269,-121.95201, distance to the offices: 2613


Near the offices there is a pet hairdresser at 2.6km

-----------------------------

# Third company analysis

In [310]:
# I make a query to the Foursquare API to look for tech startups near this company.

api_function('37.371088,-121.925355', 'Tech Startup', 'tech_startups_sj2')


{'meta': {'code': 200, 'requestId': '5fb28fcb368fe8722593687f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2229,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'tech startup',
  'totalResults': 172,
  'suggestedBounds': {'ne': {'lat': 37.377540657272796,
    'lng': -121.91772149685794},
   'sw': {'lat': 37.36705419727112, 'lng': -121.92596859824486}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51c0abff498e828260d4a5cc',
       'name': 'PayPal',
       'location': {'address': '2211 N 1st St',
        'lat': 37.375874679466456,
        'lng': -121.92306376202592,
        'l

In [250]:
print(dist(tech_startups_sj2))
print("----------------------------------------------")
print(lat(tech_startups_sj2))
print("----------------------------------------------")
print(lng(tech_startups_sj2))
print("----------------------------------------------")
print(name(tech_startups_sj2))

[570, 17, 243, 366, 396, 412, 530, 573, 21, 602, 702, 746, 749, 758]
----------------------------------------------
[37.375874679466456, 37.370972648019674, 37.36947, 37.36957044583222, 37.3695173976203, 37.369302000000005, 37.369813, 37.36805224766053, 37.37114012, 37.36753085454393, 37.377064, 37.36766592556348, 37.368907, 37.367859]
----------------------------------------------
[-121.92306376202592, -121.92549178587224, -121.92351, -121.921677387159, -121.92133577652343, -121.92127099999999, -121.919579, -121.92011894731519, -121.92559373, -121.92021896118176, -121.922804, -121.9180963651028, -121.917341, -121.917805]
----------------------------------------------
['PayPal', 'SourceN HQ', 'FixStream', 'River Meadow Software', 'Robin Systems', 'PlayPhone, Inc.', 'Fohboh Headquarters', 'Gaia Online', 'DataTech Labs Data Recovery', 'Ensighten', 'PayPal IT Helpdesk', 'Nutanix HQ', 'GEO Seminconductor', 'Usens']


In [12]:
dist_tech=tech_startups_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_tech=tech_startups_sj2["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech_startups_sj2["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj2["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj2["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_tech=tech_startups_sj2["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: SourceN HQ, company coordinates: 37.370972648019674,-121.92549178587224, distance to the offices: 17
company name: DataTech Labs Data Recovery, company coordinates: 37.37114012,-121.92559373, distance to the offices: 21


There are two tech startups very close to the company

In [309]:
# I make a query to the Foursquare API to look for design studios near this company.

api_function('37.371088,-121.925355', 'Design Studio', 'design_studio_sj2')


{'meta': {'code': 200, 'requestId': '5fb28fba33f6626c68157b50'},
 'response': {'suggestedRadius': 2229,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'design studio',
  'totalResults': 53,
  'suggestedBounds': {'ne': {'lat': 37.38488118593506, 'lng': -121.90373605},
   'sw': {'lat': 37.36479886733642, 'lng': -121.94608495000001}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '555cc5ae498e23396776d0b1',
       'name': 'Kiskon Technologies',
       'location': {'address': '2076 Gateway Pl # 500',
        'lat': 37.37334604119023,
        'lng': -121.92193508148193,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.37334604119023,
          'lng': -121.9

In [249]:
print(dist(design_studio_sj2))
print("----------------------------------------------")
print(lat(design_studio_sj2))
print("----------------------------------------------")
print(lng(design_studio_sj2))
print("----------------------------------------------")
print(name(design_studio_sj2))

[393, 929, 736, 1411, 1505, 1606, 1619, 1742, 1763, 1854, 1551, 1744, 1826, 2305]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.378426714690484, 37.3707160949707, 37.379857, 37.374013, 37.373688, 37.365846999999995, 37.383968353271484, 37.3657117, 37.3656546, 37.38424, 37.38526153564453]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.91234536963067, -121.9083480834961, -121.910925, -121.907427, -121.905933, -121.94416000000001, -121.91206359863281, -121.9091752, -121.9068635, -121.91301, -121.9063491821289]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Home Expo &  Design', 'Woodenbridge', 'Bay Stone', 'Atlas Glass', 'Granite & Stone Expo', 'All Architectural Stone', 'Emser Tile&Natural Stone', 'Signarama San Jose', 'Speck Design', '104 Octane', 'Bath Solutions by Mike Counsil']


In [16]:
dist_design=design_studio_sj2["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_design=design_studio_sj2["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_design=design_studio_sj2["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_design=design_studio_sj2["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Kiskon Technologies, company coordinates: 37.37334604119023,-121.92193508148193, distance to the offices: 393


There is one design studio near around 400m

In [308]:
# I make a query to the Foursquare API to look for schools near this company.

api_function('37.371088,-121.925355', 'Elementary School', 'elementary_sch_sj2')


{'meta': {'code': 200, 'requestId': '5fb28fa688be05604bf644ea'},
 'response': {'suggestedRadius': 2229,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'elementary school',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 37.409873762716295,
    'lng': -121.87983937403224},
   'sw': {'lat': 37.32654227934447, 'lng': -121.96274533744061}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c925989b641236a58358979',
       'name': 'Bachrodt Elementary School',
       'location': {'address': '102 Sonora Ave',
        'lat': 37.36330687882652,
        'lng': -121.9127954630632,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.36330687882652,
         

In [248]:
print(dist(elementary_sch_sj2))
print("----------------------------------------------")
print(lat(elementary_sch_sj2))
print("----------------------------------------------")
print(lng(elementary_sch_sj2))
print("----------------------------------------------")
print(name(elementary_sch_sj2))

[1408, 2784, 2874, 3343, 3399, 3065, 3782, 3579, 3993, 4106, 4464, 4537, 4577, 4650]
----------------------------------------------
[37.36330687882652, 37.38955552112065, 37.388926007147695, 37.35738251403415, 37.343393216725616, 37.347336, 37.348379, 37.398261959325076, 37.350991, 37.38721347682061, 37.333647875243265, 37.3303300740432, 37.40608596801758, 37.343032172509545]
----------------------------------------------
[-121.9127954630632, -121.94658055857437, -121.90186142921448, -121.9589768845584, -121.94153776982016, -121.942885, -121.893555, -121.94699718220039, -121.88796799999999, -121.88360782691444, -121.94342527769899, -121.92598071783651, -121.898193359375, -121.96429400003163]
----------------------------------------------
['Bachrodt Elementary School', 'Montague Elementary School', 'Orchard Elementary School', 'Scott Lane Elementary School', 'Washington Open Elementary', "St Clare's Catholic School", 'Suzume No Gakko', 'Don Callejon School', 'Grant Elementary School', '

In [19]:
dist_school=elementary_sch_sj2["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_school=elementary_sch_sj2["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_school=elementary_sch_sj2["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_school=elementary_sch_sj2["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Bachrodt Elementary School, company coordinates: 37.36330687882652,-121.9127954630632, distance to the offices: 1408


The nearest school is at 1.4km

In [307]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

api_function('37.371088,-121.925355', 'Nightlife Spot', 'nightlife_spot_sj2')


{'meta': {'code': 200, 'requestId': '5fb28f92e6b2a16984a6c3d0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2229,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'nightlife spot',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 37.37651835021626,
    'lng': -121.90555460815912},
   'sw': {'lat': 37.36012064545862, 'lng': -121.95593924550403}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d97ef6a97d06ea85146290b',
       'name': 'The Doghouse Sports Lounge',
       'location': {'address': '1336 N 1st St',
        'crossStreet': 'Rosemary',
        'lat': 37.360865995674

In [247]:
print(dist(nightlife_spot_sj2))
print("----------------------------------------------")
print(lat(nightlife_spot_sj2))
print("----------------------------------------------")
print(lng(nightlife_spot_sj2))
print("----------------------------------------------")
print(name(nightlife_spot_sj2))

[1877, 2729, 498, 1747, 587, 272, 490, 509, 283, 2448, 486, 476, 1467, 2159]
----------------------------------------------
[37.36086599567488, 37.36132088609424, 37.37124135626228, 37.36381793355914, 37.365813193961756, 37.37152348661807, 37.36690004129253, 37.37082598391839, 37.371769553739405, 37.35856138262397, 37.36684205185309, 37.36699705883182, 37.37458419799805, 37.3517028899461]
----------------------------------------------
[-121.90847504592831, -121.95364903471561, -121.91972331072563, -121.90784481894752, -121.9256078829507, -121.92232810486176, -121.92706413888456, -121.93110972058392, -121.92226542378253, -121.90260409915966, -121.92666041405536, -121.92694046682892, -121.90935516357422, -121.92446824885526]
----------------------------------------------
['The Doghouse Sports Lounge', 'Golden State Brewery', 'Zone 8 Sports Bar & Grill', '4th St Bowl Lounge', 'Santa Cruz Wine Bar', 'Doubletree Pool', 'shark cage', 'Access Point', '2050 Lobby Lounge & Bar', 'Hyde Park Loun

In [26]:

dist_party=nightlife_spot_sj2["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj2["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj2["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj2["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=nightlife_spot_sj2["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj2["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj2["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj2["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")



company name: 2050 Lobby Lounge & Bar, company coordinates: 37.371769553739405,-121.92226542378253, distance to the offices: 283
company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 272


The nearest nighlife spots are at 300m approximately

In [306]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

api_function('37.371088,-121.925355', 'Vegan Restaurant', 'veg_rest_sj2')


{'meta': {'code': 200, 'requestId': '5fb28f7b164c641f7b3291b4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2229,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'vegan restaurant',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 37.42560895276847,
    'lng': -121.88480138993586},
   'sw': {'lat': 37.32140595492598, 'lng': -121.9817248113471}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '507e209ae4b0959d8bc30ecc',
       'name': "Ike's Sandwiches",
       'location': {'address': '2235 The Alameda',
        'crossStreet': 'at Chapman Ct',
        'lat': 37.3446038364260

In [246]:
print(dist(veg_rest_sj2))
print("----------------------------------------------")
print(lat(veg_rest_sj2))
print("----------------------------------------------")
print(lng(veg_rest_sj2))
print("----------------------------------------------")
print(name(veg_rest_sj2))

[3022, 3335, 5277, 2217, 2501, 2582, 3420, 3719, 3901, 4114, 4257, 5065, 4786, 5600]
----------------------------------------------
[37.34460383642609, 37.343718446032305, 37.32614245482791, 37.385706, 37.35659, 37.3607916, 37.34049606323242, 37.33930772, 37.338702000000005, 37.404856, 37.396342, 37.35198682571322, 37.41086084497713, 37.42087245286653]
----------------------------------------------
[-121.93286117841694, -121.9406861466985, -121.94432964084655, -121.908328, -121.94696, -121.8991962, -121.92169952392578, -121.93835143, -121.942205, -121.906444, -121.88920700000001, -121.97731920128297, -121.94593038741806, -121.91625220186718]
----------------------------------------------
["Ike's Sandwiches", "deedee's", 'Loving Hut', 'Vegefarm', 'The Happy Hooligans', "Brother Baby's BBQ", 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie', 'Tasty Vegetarian Cuisine', 'SJ Vegetarian', 'Peacock Indian Cuisine', "Sri's Kitchen", 'Loving Hut']


In [31]:
dist_vegan=veg_rest_sj2["response"]["groups"][0]["items"][3]["venue"]["location"]["distance"]
lat_vegan=veg_rest_sj2["response"]["groups"][0]["items"][3]["venue"]["location"]["lat"]
lng_vegan=veg_rest_sj2["response"]["groups"][0]["items"][3]["venue"]["location"]["lng"]
name_vegan=veg_rest_sj2["response"]["groups"][0]["items"][3]["venue"]["name"]

print(f"company name: {name_vegan}, company coordinates: {lat_vegan},{lng_vegan}, distance to the offices: {dist_vegan}")

company name: Brother Baby's BBQ, company coordinates: 37.3607916,-121.8991962, distance to the offices: 2582


The nearest one is at 2.5km

In [305]:
# I make a query to the Foursquare API to look for basketball courts near this company.

api_function('37.371088,-121.925355', 'Basketball Court', 'basketball_court_sj2')


{'meta': {'code': 200, 'requestId': '5fb28f64aa0cc20f98ec0802'},
 'response': {'suggestedRadius': 2229,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'basketball court',
  'totalResults': 19,
  'suggestedBounds': {'ne': {'lat': 37.420003872877636,
    'lng': -121.87698936462401},
   'sw': {'lat': 37.32647170912039, 'lng': -121.97387886047365}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a2f2f67f964a520b3981fe3',
       'name': 'Central YMCA',
       'location': {'address': '1717 The Alameda',
        'crossStreet': 'at Naglee Ave.',
        'lat': 37.33779578622328,
        'lng': -121.91989631992244,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [242]:
print(dist(basketball_court_sj2))
print("----------------------------------------------")
print(lat(basketball_court_sj2))
print("----------------------------------------------")
print(lng(basketball_court_sj2))
print("----------------------------------------------")
print(name(basketball_court_sj2))

[3737, 695, 1160, 1231, 1250, 2637, 2865, 3475, 3810, 4573, 5035, 4136, 5514, 4619]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.37117385864258, 37.364402, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.40387815016547, 37.41575241088867, 37.393019101339256, 37.35904935160653, 37.356725437277845]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.91143798828125, -121.913997, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.9565124356151, -121.91634368896484, -121.96311097120844, -121.98581956228077, -121.87637085671278]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Basketball Gym', 'Skyport!', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Lick Mill Park', 'Cisco - Basketball Court', 'Agnew Park', '

In [34]:
dist_basket=basketball_court_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basketball_court_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basketball_court_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basketball_court_sj2["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 695


There is one basketball court at 700m

In [304]:
# I make a query to the Foursquare API to look for Starbucks near this company.

api_function('37.371088,-121.925355', 'starbucks', 'starbucks_sj2')


{'meta': {'code': 200, 'requestId': '5fb28f46f6c3124343ad23be'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2229,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'starbucks',
  'totalResults': 50,
  'suggestedBounds': {'ne': {'lat': 37.4100811785, 'lng': -121.90288023178267},
   'sw': {'lat': 37.3305692515, 'lng': -121.9547951325636}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a6b4b07f964a5205cce1fe3',
       'name': 'Starbucks',
       'location': {'address': '1751 N 1st St',
        'lat': 37.37092547570862,
        'lng': -121.91

In [241]:
print(dist(starbucks_sj2))
print("----------------------------------------------")
print(lat(starbucks_sj2))
print("----------------------------------------------")
print(lng(starbucks_sj2))
print("----------------------------------------------")
print(name(starbucks_sj2))

[715, 254, 405, 1154, 482, 2298, 3032, 4109, 1607, 3216, 3017, 3517, 3671, 3413]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.35060173, 37.345295, 37.33418343, 37.385382, 37.395157, 37.398163, 37.341431, 37.40197092074824, 37.39293751646301]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9221276, -121.936371, -121.9241052, -121.927919, -121.90523999999999, -121.923706, -121.911642, -121.93991940591599, -121.95243536434629]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [38]:

dist_starbucks=starbucks_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sj2["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks_sj2["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 254


Near the company there is a Starbucks at 254m

In [303]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

api_function('37.371088,-121.925355', 'Pet Service', 'pet_service_sj2')


{'meta': {'code': 200, 'requestId': '5fb28f294769b850712509a3'},
 'response': {'suggestedRadius': 2229,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'pet service',
  'totalResults': 31,
  'suggestedBounds': {'ne': {'lat': 37.40474107281321,
    'lng': -121.8918994965814},
   'sw': {'lat': 37.32485747092281, 'lng': -121.98063877179027}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '49f6928cf964a520506c1fe3',
       'name': 'PetSmart',
       'location': {'address': '607 Coleman Ave',
        'crossStreet': 'at W Taylor St',
        'lat': 37.339942,
        'lng': -121.907791,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.339942,
          'lng':

In [240]:
print(dist(pet_service_sj2))
print("----------------------------------------------")
print(lat(pet_service_sj2))
print("----------------------------------------------")
print(lng(pet_service_sj2))
print("----------------------------------------------")
print(name(pet_service_sj2))

[3799, 2843, 4897, 3766, 4958, 2536, 2586, 2686, 2869, 3225, 3390, 3600, 3606, 3622]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.34972481944317, 37.38021, 37.40111, 37.34043631759206, 37.343559452656756, 37.343485]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.90721511840819, -121.95995900000001, -121.91887, -121.91236495971678, -121.94683830386161, -121.947041]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Wag Hotels', 'Epic Dog Park', 'Primal Canine', "Juan's Groom Room", 'The Tails In The City']


In [43]:
dist_dog=pet_service_sj2["response"]["groups"][0]["items"][6]["venue"]["location"]["distance"]
lat_dog=pet_service_sj2["response"]["groups"][0]["items"][6]["venue"]["location"]["lat"]
lng_dog=pet_service_sj2["response"]["groups"][0]["items"][6]["venue"]["location"]["lng"]
name_dog=pet_service_sj2["response"]["groups"][0]["items"][6]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")

company name: Bay Area K9 Association LLC., company coordinates: 37.36273680000001,-121.9526368, distance to the offices: 2586


The pet hairdresser is at 2.5km from the company

-------

# Fourth company analysis

In [302]:
# I make a query to the Foursquare API to look for tech startups near this company.

api_function('37.3709977,-121.9235529', 'Tech Startup', 'tech_startups_sj3')


{'meta': {'code': 200, 'requestId': '5fb28f0be2d2433c2133811c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'tech startup',
  'totalResults': 173,
  'suggestedBounds': {'ne': {'lat': 37.37629187071258,
    'lng': -121.91692836349999},
   'sw': {'lat': 37.3671136632978, 'lng': -121.9260063665}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51c0abff498e828260d4a5cc',
       'name': 'PayPal',
       'location': {'address': '2211 N 1st St',
        'lat': 37.375874679466456,
        'lng': -121.92306376202592,
        'labeled

In [239]:
print(dist(tech_startups_sj3))
print("----------------------------------------------")
print(lat(tech_startups_sj3))
print("----------------------------------------------")
print(lng(tech_startups_sj3))
print("----------------------------------------------")
print(name(tech_startups_sj3))

[544, 171, 170, 229, 256, 276, 446, 485, 596, 181, 608, 616, 634, 678]
----------------------------------------------
[37.375874679466456, 37.370972648019674, 37.36947, 37.36957044583222, 37.3695173976203, 37.369302000000005, 37.36805224766053, 37.36753085454393, 37.368907, 37.37114012, 37.36766592556348, 37.367859, 37.368927, 37.377064]
----------------------------------------------
[-121.92306376202592, -121.92549178587224, -121.92351, -121.921677387159, -121.92133577652343, -121.92127099999999, -121.92011894731519, -121.92021896118176, -121.917341, -121.92559373, -121.9180963651028, -121.917805, -121.91687, -121.922804]
----------------------------------------------
['PayPal', 'SourceN HQ', 'FixStream', 'River Meadow Software', 'Robin Systems', 'PlayPhone, Inc.', 'Gaia Online', 'Ensighten', 'GEO Seminconductor', 'DataTech Labs Data Recovery', 'Nutanix HQ', 'Usens', 'XtremIO', 'PayPal IT Helpdesk']


In [52]:
dist_tech=tech_startups_sj3["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj3["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj3["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_tech=tech_startups_sj3["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech_startups_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_tech=tech_startups_sj3["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech_startups_sj3["response"]["groups"][0]["items"][9]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj3["response"]["groups"][0]["items"][9]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj3["response"]["groups"][0]["items"][9]["venue"]["location"]["lng"]
name_tech=tech_startups_sj3["response"]["groups"][0]["items"][9]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: FixStream, company coordinates: 37.36947,-121.92351, distance to the offices: 170
company name: SourceN HQ, company coordinates: 37.370972648019674,-121.92549178587224, distance to the offices: 171
company name: DataTech Labs Data Recovery, company coordinates: 37.37114012,-121.92559373, distance to the offices: 181


There are tree tech companies very close

In [301]:
# I make a query to the Foursquare API to look for design studios near this company.

api_function('37.3709977,-121.9235529', 'Design Studio', 'design_studio_sj3')


{'meta': {'code': 200, 'requestId': '5fb28ef6f32ddb151e5e3141'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'design studio',
  'totalResults': 55,
  'suggestedBounds': {'ne': {'lat': 37.38488118593506, 'lng': -121.90373605},
   'sw': {'lat': 37.36479886733642, 'lng': -121.94608495000001}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '555cc5ae498e23396776d0b1',
       'name': 'Kiskon Technologies',
       'location': {'address': '2076 Gateway Pl # 500',
        'lat': 37.37334604119023,
        'lng': -121.92193508148193,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.37334604119023,
          'lng': -121.9

In [238]:
print(dist(design_studio_sj3))
print("----------------------------------------------")
print(lat(design_studio_sj3))
print("----------------------------------------------")
print(lng(design_studio_sj3))
print("----------------------------------------------")
print(name(design_studio_sj3))

[298, 785, 582, 1291, 1345, 1465, 1490, 1587, 1765, 1401, 1911, 1591, 1744, 2199]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.378426714690484, 37.3707160949707, 37.374013, 37.379857, 37.373688, 37.383968353271484, 37.3657117, 37.365846999999995, 37.3656546, 37.38424, 37.38526153564453]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.91234536963067, -121.9083480834961, -121.907427, -121.910925, -121.905933, -121.91206359863281, -121.9091752, -121.94416000000001, -121.9068635, -121.91301, -121.9063491821289]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Home Expo &  Design', 'Woodenbridge', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Emser Tile&Natural Stone', 'Signarama San Jose', 'All Architectural Stone', 'Speck Design', '104 Octane', 'Bath Solutions by Mike Counsil']


In [56]:
dist_design=design_studio_sj3["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_design=design_studio_sj3["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_design=design_studio_sj3["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_design=design_studio_sj3["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Kiskon Technologies, company coordinates: 37.37334604119023,-121.92193508148193, distance to the offices: 298


There is one design studio at 300m from the offices

In [300]:
# I make a query to the Foursquare API to look for schools near this company.

api_function('37.3709977,-121.9235529', 'Elementary School', 'elementary_sch_sj3')


{'meta': {'code': 200, 'requestId': '5fb28edbe6b2a16984a41dc1'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'elementary school',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 37.409873762716295,
    'lng': -121.87983937403224},
   'sw': {'lat': 37.32654227934447, 'lng': -121.96274533744061}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c925989b641236a58358979',
       'name': 'Bachrodt Elementary School',
       'location': {'address': '102 Sonora Ave',
        'lat': 37.36330687882652,
        'lng': -121.9127954630632,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.36330687882652,
         

In [237]:
print(dist(elementary_sch_sj3))
print("----------------------------------------------")
print(lat(elementary_sch_sj3))
print("----------------------------------------------")
print(lng(elementary_sch_sj3))
print("----------------------------------------------")
print(name(elementary_sch_sj3))

[1280, 2768, 2901, 3460, 3481, 3140, 3658, 3856, 3967, 3675, 4504, 4514, 4532, 4560]
----------------------------------------------
[37.36330687882652, 37.388926007147695, 37.38955552112065, 37.343393216725616, 37.35738251403415, 37.347336, 37.348379, 37.350991, 37.38721347682061, 37.398261959325076, 37.40608596801758, 37.333647875243265, 37.3303300740432, 37.35531655464134]
----------------------------------------------
[-121.9127954630632, -121.90186142921448, -121.94658055857437, -121.94153776982016, -121.9589768845584, -121.942885, -121.893555, -121.88796799999999, -121.88360782691444, -121.94699718220039, -121.898193359375, -121.94342527769899, -121.92598071783651, -121.87593729046975]
----------------------------------------------
['Bachrodt Elementary School', 'Orchard Elementary School', 'Montague Elementary School', 'Washington Open Elementary', 'Scott Lane Elementary School', "St Clare's Catholic School", 'Suzume No Gakko', 'Grant Elementary School', 'Willow Glen Elementary S

In [60]:
dist_school=elementary_sch_sj3["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_school=elementary_sch_sj3["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_school=elementary_sch_sj3["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_school=elementary_sch_sj3["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Bachrodt Elementary School, company coordinates: 37.36330687882652,-121.9127954630632, distance to the offices: 1280


The nearest school is at 1.2km

In [299]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

api_function('37.3709977,-121.9235529', 'Nightlife Spot', 'nightlife_spot_sj3')


{'meta': {'code': 200, 'requestId': '5fb28ec68d3f8f53d3d14d37'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'nightlife spot',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 37.376633580868805,
    'lng': -121.90005185238185},
   'sw': {'lat': 37.357700801755165, 'lng': -121.95620128149342}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d97ef6a97d06ea85146290b',
       'name': 'The Doghouse Sports Lounge',
       'location': {'address': '1336 N 1st St',
        'crossStreet': 'Rosemary',
        'lat': 37.3608659956

In [236]:
print(dist(nightlife_spot_sj3))
print("----------------------------------------------")
print(lat(nightlife_spot_sj3))
print("----------------------------------------------")
print(lng(nightlife_spot_sj3))
print("----------------------------------------------")
print(name(nightlife_spot_sj3))

[1746, 2872, 339, 1603, 605, 123, 551, 668, 2313, 142, 538, 1317, 536, 1522]
----------------------------------------------
[37.36086599567488, 37.36132088609424, 37.37124135626228, 37.36381793355914, 37.365813193961756, 37.37152348661807, 37.36690004129253, 37.37082598391839, 37.35856138262397, 37.371769553739405, 37.36684205185309, 37.37458419799805, 37.36699705883182, 37.364403]
----------------------------------------------
[-121.90847504592831, -121.95364903471561, -121.91972331072563, -121.90784481894752, -121.9256078829507, -121.92232810486176, -121.92706413888456, -121.93110972058392, -121.90260409915966, -121.92226542378253, -121.92666041405536, -121.90935516357422, -121.92694046682892, -121.908478]
----------------------------------------------
['The Doghouse Sports Lounge', 'Golden State Brewery', 'Zone 8 Sports Bar & Grill', '4th St Bowl Lounge', 'Santa Cruz Wine Bar', 'Doubletree Pool', 'shark cage', 'Access Point', 'Hyde Park Lounge', '2050 Lobby Lounge & Bar', 'The Brit'

In [66]:
dist_party=nightlife_spot_sj3["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj3["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj3["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj3["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=nightlife_spot_sj3["response"]["groups"][0]["items"][9]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj3["response"]["groups"][0]["items"][9]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj3["response"]["groups"][0]["items"][9]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj3["response"]["groups"][0]["items"][9]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 123
company name: 2050 Lobby Lounge & Bar, company coordinates: 37.371769553739405,-121.92226542378253, distance to the offices: 142


There are two places to go to party near the offices

In [298]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

api_function('37.3709977,-121.9235529', 'Vegan Restaurant', 'veg_rest_sj3')


{'meta': {'code': 200, 'requestId': '5fb28ead32abda5e0875cb22'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'vegan restaurant',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 37.42560895276847,
    'lng': -121.88480138993586},
   'sw': {'lat': 37.32140595492598, 'lng': -121.9817248113471}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '507e209ae4b0959d8bc30ecc',
       'name': "Ike's Sandwiches",
       'location': {'address': '2235 The Alameda',
        'crossStreet': 'at Chapman Ct',
        'lat': 37.3446038364260

In [235]:
print(dist(veg_rest_sj3))
print("----------------------------------------------")
print(lat(veg_rest_sj3))
print("----------------------------------------------")
print(lng(veg_rest_sj3))
print("----------------------------------------------")
print(name(veg_rest_sj3))

[3051, 3394, 5320, 2120, 2619, 2436, 3399, 3762, 3955, 4061, 4145, 4858, 5206, 5589]
----------------------------------------------
[37.34460383642609, 37.343718446032305, 37.32614245482791, 37.385706, 37.35659, 37.3607916, 37.34049606323242, 37.33930772, 37.338702000000005, 37.404856, 37.396342, 37.41086084497713, 37.35198682571322, 37.42087245286653]
----------------------------------------------
[-121.93286117841694, -121.9406861466985, -121.94432964084655, -121.908328, -121.94696, -121.8991962, -121.92169952392578, -121.93835143, -121.942205, -121.906444, -121.88920700000001, -121.94593038741806, -121.97731920128297, -121.91625220186718]
----------------------------------------------
["Ike's Sandwiches", "deedee's", 'Loving Hut', 'Vegefarm', 'The Happy Hooligans', "Brother Baby's BBQ", 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie', 'Tasty Vegetarian Cuisine', 'SJ Vegetarian', "Sri's Kitchen", 'Peacock Indian Cuisine', 'Loving Hut']


In [71]:
dist_vegan=veg_rest_sj3["response"]["groups"][0]["items"][3]["venue"]["location"]["distance"]
lat_vegan=veg_rest_sj3["response"]["groups"][0]["items"][3]["venue"]["location"]["lat"]
lng_vegan=veg_rest_sj3["response"]["groups"][0]["items"][3]["venue"]["location"]["lng"]
name_vegan=veg_rest_sj3["response"]["groups"][0]["items"][3]["venue"]["name"]

print(f"company name: {name_vegan}, company coordinates: {lat_vegan},{lng_vegan}, distance to the offices: {dist_vegan}")

company name: Brother Baby's BBQ, company coordinates: 37.3607916,-121.8991962, distance to the offices: 2436


The nearest vegan restaurant is at 2.4km

In [297]:
# I make a query to the Foursquare API to look for basketball courts near this company.

api_function('37.3709977,-121.9235529', 'Basketball Court', 'basketball_court_sj3')


{'meta': {'code': 200, 'requestId': '5fb28e955111134f6b34b2c6'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'basketball court',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 37.420003872877636,
    'lng': -121.87698936462401},
   'sw': {'lat': 37.32647170912039, 'lng': -121.97387886047365}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a2f2f67f964a520b3981fe3',
       'name': 'Central YMCA',
       'location': {'address': '1717 The Alameda',
        'crossStreet': 'at Naglee Ave.',
        'lat': 37.33779578622328,
        'lng': -121.91989631992244,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [231]:
print(dist(basketball_court_sj3))
print("----------------------------------------------")
print(lat(basketball_court_sj3))
print("----------------------------------------------")
print(lng(basketball_court_sj3))
print("----------------------------------------------")
print(name(basketball_court_sj3))

[3710, 668, 1036, 1071, 1119, 2757, 3022, 3556, 3970, 4679, 5022, 5159, 4272, 5667]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.37117385864258, 37.364402, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.40387815016547, 37.41575241088867, 37.35042190551758, 37.393019101339256, 37.35904935160653]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.91143798828125, -121.913997, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.9565124356151, -121.91634368896484, -121.87129974365234, -121.96311097120844, -121.98581956228077]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Basketball Gym', 'Skyport!', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Lick Mill Park', 'Cisco - Basketball Court', 'San Jose High S

In [74]:
dist_basket=basketball_court_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basketball_court_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basketball_court_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basketball_court_sj3["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 668


The basketball court is at 700m from the offices

In [296]:
# I make a query to the Foursquare API to look for Starbucks near this company.

api_function('37.3709977,-121.9235529', 'starbucks', 'starbucks_sj3')


{'meta': {'code': 200, 'requestId': '5fb28e7e334c6705654590b2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'starbucks',
  'totalResults': 50,
  'suggestedBounds': {'ne': {'lat': 37.4100811785, 'lng': -121.89204880000001},
   'sw': {'lat': 37.3305692515, 'lng': -121.94412720000001}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a6b4b07f964a5205cce1fe3',
       'name': 'Starbucks',
       'location': {'address': '1751 N 1st St',
        'lat': 37.37092547570862,
        'lng': -121.9

In [229]:
print(dist(starbucks_sj3))
print("----------------------------------------------")
print(lat(starbucks_sj3))
print("----------------------------------------------")
print(lng(starbucks_sj3))
print("----------------------------------------------")
print(name(starbucks_sj3))

[555, 110, 550, 1196, 340, 2273, 4098, 3077, 3139, 1647, 3024, 3455, 3739, 4264]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.35060173, 37.33418343, 37.345295, 37.395157, 37.385382, 37.398163, 37.341431, 37.40197092074824, 37.406467]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9221276, -121.9241052, -121.936371, -121.90523999999999, -121.927919, -121.923706, -121.911642, -121.93991940591599, -121.94176]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [81]:
dist_starbucks=starbucks_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sj3["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks_sj3["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

dist_starbucks=starbucks_sj3["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sj3["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sj3["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_starbucks=starbucks_sj3["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 110
company name: Starbucks, company coordinates: 37.369186,-121.92045, distance to the offices: 340


There are two Starbucks near

In [295]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

api_function('37.3709977,-121.9235529', 'Pet Service', 'pet_service_sj3')


{'meta': {'code': 200, 'requestId': '5fb28e6e0cc1d6096f441133'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'pet service',
  'totalResults': 33,
  'suggestedBounds': {'ne': {'lat': 37.40474107281321,
    'lng': -121.88153774475427},
   'sw': {'lat': 37.32485747092281, 'lng': -121.98113218854394}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '49f6928cf964a520506c1fe3',
       'name': 'PetSmart',
       'location': {'address': '607 Coleman Ave',
        'crossStreet': 'at W Taylor St',
        'lat': 37.339942,
        'lng': -121.907791,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.339942,
          'lng'

In [230]:
print(dist(pet_service_sj3))
print("----------------------------------------------")
print(lat(pet_service_sj3))
print("----------------------------------------------")
print(lng(pet_service_sj3))
print("----------------------------------------------")
print(name(pet_service_sj3))

[3727, 2702, 4849, 3694, 5100, 2682, 2732, 2839, 2774, 3377, 3379, 3543, 3424, 3684]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.34972481944317, 37.40111, 37.38021, 37.34043631759206, 37.363336300242906, 37.343559452656756]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.90721511840819, -121.91887, -121.95995900000001, -121.91236495971678, -121.8860647649265, -121.94683830386161]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Epic Dog Park', 'Wag Hotels', 'Primal Canine', 'San Jose Animal Hospital', "Juan's Groom Room"]


In [88]:
dist_dog=pet_service_sj3["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_dog=pet_service_sj3["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_dog=pet_service_sj3["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_dog=pet_service_sj3["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")

company name: Ohana Kennels, company coordinates: 37.36269,-121.95201, distance to the offices: 2682


The pet hairdresser is at 2.6km from the company

-----

# Fifth company analysis

In [294]:
# I make a query to the Foursquare API to look for tech startups near this company.

api_function('37.3706126,-121.9219538', 'Tech Startup', 'tech_startups_sj4')


{'meta': {'code': 200, 'requestId': '5fb28e4ed64a7347259f864a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'tech startup',
  'totalResults': 178,
  'suggestedBounds': {'ne': {'lat': 37.37629187071258,
    'lng': -121.9164389107064},
   'sw': {'lat': 37.3671136632978, 'lng': -121.92592287516584}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51c0abff498e828260d4a5cc',
       'name': 'PayPal',
       'location': {'address': '2211 N 1st St',
        'lat': 37.375874679466456,
        'lng': -121.92306376202592,
        'labe

In [227]:
print(dist(tech_startups_sj4))
print("----------------------------------------------")
print(lat(tech_startups_sj4))
print("----------------------------------------------")
print(lng(tech_startups_sj4))
print("----------------------------------------------")
print(name(tech_startups_sj4))

[593, 157, 118, 133, 187, 315, 328, 375, 450, 473, 478, 487, 560, 576]
----------------------------------------------
[37.375874679466456, 37.369302000000005, 37.36957044583222, 37.3695173976203, 37.36947, 37.370972648019674, 37.36805224766053, 37.36753085454393, 37.368907, 37.36766592556348, 37.367859, 37.368927, 37.36666122235786, 37.36609659285964]
----------------------------------------------
[-121.92306376202592, -121.92127099999999, -121.921677387159, -121.92133577652343, -121.92351, -121.92549178587224, -121.92011894731519, -121.92021896118176, -121.917341, -121.9180963651028, -121.917805, -121.91687, -121.91802978515625, -121.91876875122978]
----------------------------------------------
['PayPal', 'PlayPhone, Inc.', 'River Meadow Software', 'Robin Systems', 'FixStream', 'SourceN HQ', 'Gaia Online', 'Ensighten', 'GEO Seminconductor', 'Nutanix HQ', 'Usens', 'XtremIO', 'Ziffity Solutions LLC', 'Calypto Design Systems']


In [99]:
dist_tech=tech_startups_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_tech=tech_startups_sj4["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech_startups_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_tech=tech_startups_sj4["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")


dist_tech=tech_startups_sj4["response"]["groups"][0]["items"][3]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj4["response"]["groups"][0]["items"][3]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj4["response"]["groups"][0]["items"][3]["venue"]["location"]["lng"]
name_tech=tech_startups_sj4["response"]["groups"][0]["items"][3]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

dist_tech=tech_startups_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_tech=tech_startups_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_tech=tech_startups_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_tech=tech_startups_sj4["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_tech}, company coordinates: {lat_tech},{lng_tech}, distance to the offices: {dist_tech}")

company name: PlayPhone, Inc., company coordinates: 37.369302000000005,-121.92127099999999, distance to the offices: 157
company name: River Meadow Software, company coordinates: 37.36957044583222,-121.921677387159, distance to the offices: 118
company name: Robin Systems, company coordinates: 37.3695173976203,-121.92133577652343, distance to the offices: 133
company name: FixStream, company coordinates: 37.36947,-121.92351, distance to the offices: 187


There are 4 tech startups very close to the offices

In [293]:
# I make a query to the Foursquare API to look for design studios near this company.

api_function('37.3706126,-121.9219538', 'Design Studio', 'design_studio_sj4')


{'meta': {'code': 200, 'requestId': '5fb28e3a3023093e62660671'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'design studio',
  'totalResults': 60,
  'suggestedBounds': {'ne': {'lat': 37.384884040935056,
    'lng': -121.9048472959259},
   'sw': {'lat': 37.36473891233643, 'lng': -121.92274878555604}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '555cc5ae498e23396776d0b1',
       'name': 'Kiskon Technologies',
       'location': {'address': '2076 Gateway Pl # 500',
        'lat': 37.37334604119023,
        'lng': -121.92193508148193,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.37334604119023,
          'ln

In [226]:
print(dist(design_studio_sj4))
print("----------------------------------------------")
print(lat(design_studio_sj4))
print("----------------------------------------------")
print(lng(design_studio_sj4))
print("----------------------------------------------")
print(name(design_studio_sj4))

[304, 643, 434, 1134, 1203, 1216, 1231, 1231, 1339, 1418, 1458, 1485, 1255, 1725]
----------------------------------------------
[37.37334604119023, 37.3672899, 37.3691219, 37.37102127075195, 37.3707160949707, 37.378426714690484, 37.374528, 37.373359, 37.374013, 37.379857, 37.373688, 37.373843, 37.3657117, 37.383968353271484]
----------------------------------------------
[-121.92193508148193, -121.916, -121.917411, -121.90914154052734, -121.9083480834961, -121.91234536963067, -121.908934, -121.90847, -121.907427, -121.910925, -121.905933, -121.905661, -121.9091752, -121.91206359863281]
----------------------------------------------
['Kiskon Technologies', 'Watry Design', 'Konica Minolta Business Solutions', 'Screened Magic', 'Woodenbridge', 'Home Expo &  Design', 'Summit Cabinet', 'The Ceramic Corporation', 'Atlas Glass', 'Bay Stone', 'Granite & Stone Expo', 'Zeus Tile', 'Signarama San Jose', 'Emser Tile&Natural Stone']


In [103]:
dist_design=design_studio_sj4["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_design=design_studio_sj4["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_design=design_studio_sj4["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_design=design_studio_sj4["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

dist_design=design_studio_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_design=design_studio_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_design=design_studio_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_design=design_studio_sj4["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_design}, company coordinates: {lat_design},{lng_design}, distance to the offices: {dist_design}")

company name: Kiskon Technologies, company coordinates: 37.37334604119023,-121.92193508148193, distance to the offices: 304
company name: Konica Minolta Business Solutions, company coordinates: 37.3691219,-121.917411, distance to the offices: 434


The nearest studios are at 300m and 400m

In [292]:
# I make a query to the Foursquare API to look for schools near this company.

api_function('37.3706126,-121.9219538', 'Elementary School', 'elementary_sch_sj4')


{'meta': {'code': 200, 'requestId': '5fb28e224450ba020e9b0206'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'elementary school',
  'totalResults': 30,
  'suggestedBounds': {'ne': {'lat': 37.40922060558218,
    'lng': -121.86862555577208},
   'sw': {'lat': 37.340258579161016, 'lng': -121.96327932878633}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c925989b641236a58358979',
       'name': 'Bachrodt Elementary School',
       'location': {'address': '102 Sonora Ave',
        'lat': 37.36330687882652,
        'lng': -121.9127954630632,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.36330687882652,
         

In [225]:
print(dist(elementary_sch_sj4))
print("----------------------------------------------")
print(lat(elementary_sch_sj4))
print("----------------------------------------------")
print(lng(elementary_sch_sj4))
print("----------------------------------------------")
print(name(elementary_sch_sj4))

[1148, 2704, 3031, 3349, 3474, 3490, 3526, 3184, 3591, 3716, 3862, 3792, 4413, 4457]
----------------------------------------------
[37.36330687882652, 37.388926007147695, 37.38955552112065, 37.39309384415278, 37.358551025390625, 37.343393216725616, 37.348379, 37.347336, 37.35738251403415, 37.350991, 37.38721347682061, 37.398261959325076, 37.35531655464134, 37.379878999999995]
----------------------------------------------
[-121.9127954630632, -121.90186142921448, -121.94658055857437, -121.94713050142707, -121.95817565917969, -121.94153776982016, -121.893555, -121.942885, -121.9589768845584, -121.88796799999999, -121.88360782691444, -121.94699718220039, -121.87593729046975, -121.872928]
----------------------------------------------
['Bachrodt Elementary School', 'Orchard Elementary School', 'Montague Elementary School', 'Don Callejon School', 'Jano Hindi Bee', 'Washington Open Elementary', 'Suzume No Gakko', "St Clare's Catholic School", 'Scott Lane Elementary School', 'Grant Elementa

In [106]:
dist_school=elementary_sch_sj4["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
lat_school=elementary_sch_sj4["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
lng_school=elementary_sch_sj4["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]
name_school=elementary_sch_sj4["response"]["groups"][0]["items"][0]["venue"]["name"]

print(f"company name: {name_school}, company coordinates: {lat_school},{lng_school}, distance to the offices: {dist_school}")

company name: Bachrodt Elementary School, company coordinates: 37.36330687882652,-121.9127954630632, distance to the offices: 1148


There is a school at 1.1km from the office

In [291]:
# I make a query to the Foursquare API to look for nighlife spots near this company.

api_function('37.3706126,-121.9219538', 'Nightlife Spot', 'nightlife_spot_sj4')


{'meta': {'code': 200, 'requestId': '5fb28e0c66505c5f6ffe06ba'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'nightlife spot',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': 37.376633580868805,
    'lng': -121.90116580723543},
   'sw': {'lat': 37.357700801755165, 'lng': -121.93280822956848}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d97ef6a97d06ea85146290b',
       'name': 'The Doghouse Sports Lounge',
       'location': {'address': '1336 N 1st St',
        'crossStreet': 'Rosemary',
        'lat': 37.3608659956

In [224]:
print(dist(nightlife_spot_sj4))
print("----------------------------------------------")
print(lat(nightlife_spot_sj4))
print("----------------------------------------------")
print(lng(nightlife_spot_sj4))
print("----------------------------------------------")
print(name(nightlife_spot_sj4))

[1612, 209, 1459, 624, 106, 103, 560, 592, 2174, 612, 612, 644, 657, 671]
----------------------------------------------
[37.36086599567488, 37.37124135626228, 37.36381793355914, 37.365813193961756, 37.37152348661807, 37.371532546100354, 37.375498869419765, 37.375773, 37.35856138262397, 37.366398, 37.36690004129253, 37.36545102537579, 37.3700879, 37.369526]
----------------------------------------------
[-121.90847504592831, -121.91972331072563, -121.90784481894752, -121.9256078829507, -121.92232810486176, -121.9220969824811, -121.92348539829254, -121.923609, -121.90260409915966, -121.92641, -121.92706413888456, -121.9186465104381, -121.9145503, -121.929421]
----------------------------------------------
['The Doghouse Sports Lounge', 'Zone 8 Sports Bar & Grill', '4th St Bowl Lounge', 'Santa Cruz Wine Bar', 'Doubletree Pool', 'Fortyeight at DoubleTree', 'GSPS Wine Friday', 'Please Be Thirsty', 'Hyde Park Lounge', 'T Baggers', 'shark cage', 'the bistro', "The Arms' Ride", 'San Jose Beer

In [109]:
dist_party=nightlife_spot_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj4["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=nightlife_spot_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj4["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

dist_party=nightlife_spot_sj4["response"]["groups"][0]["items"][5]["venue"]["location"]["distance"]
lat_party=nightlife_spot_sj4["response"]["groups"][0]["items"][5]["venue"]["location"]["lat"]
lng_party=nightlife_spot_sj4["response"]["groups"][0]["items"][5]["venue"]["location"]["lng"]
name_party=nightlife_spot_sj4["response"]["groups"][0]["items"][5]["venue"]["name"]

print(f"company name: {name_party}, company coordinates: {lat_party},{lng_party}, distance to the offices: {dist_party}")

company name: Zone 8 Sports Bar & Grill, company coordinates: 37.37124135626228,-121.91972331072563, distance to the offices: 209
company name: Doubletree Pool, company coordinates: 37.37152348661807,-121.92232810486176, distance to the offices: 106
company name: Fortyeight at DoubleTree, company coordinates: 37.371532546100354,-121.9220969824811, distance to the offices: 103


There are 3 nighlife spots near the offices

In [290]:
# I make a query to the Foursquare API to look for vegetarian restaurants near this company.

api_function('37.3706126,-121.9219538', 'Vegan Restaurant', 'veg_rest_sj4')



{'meta': {'code': 200, 'requestId': '5fb28df133f6626c680ede70'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'vegan restaurant',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 37.408791677258606,
    'lng': -121.87578680608625},
   'sw': {'lat': 37.322206777569306, 'lng': -121.9503491997102}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '507e209ae4b0959d8bc30ecc',
       'name': "Ike's Sandwiches",
       'location': {'address': '2235 The Alameda',
        'crossStreet': 'at Chapman Ct',
        'lat': 37.34460383642

In [223]:
print(dist(veg_rest_sj4))
print("----------------------------------------------")
print(lat(veg_rest_sj4))
print("----------------------------------------------")
print(lng(veg_rest_sj4))
print("----------------------------------------------")
print(name(veg_rest_sj4))

[3051, 4821, 4733, 4987, 3422, 4892, 5484, 5331, 2067, 2291, 2707, 3352, 3774, 3978]
----------------------------------------------
[37.34460383642609, 37.33547578302974, 37.336852209398536, 37.341433051588915, 37.343718446032305, 37.33993504196706, 37.330296164668816, 37.32614245482791, 37.385706, 37.3607916, 37.35659, 37.34049606323242, 37.33930772, 37.338702000000005]
----------------------------------------------
[-121.93286117841694, -121.89009385118939, -121.88943603809092, -121.87917600579642, -121.9406861466985, -121.88235410455921, -121.88632717974279, -121.94432964084655, -121.908328, -121.8991962, -121.94696, -121.92169952392578, -121.93835143, -121.942205]
----------------------------------------------
["Ike's Sandwiches", 'Good Karma', "Ike's Sandwiches", 'Vegetarian House', "deedee's", 'Tofoo Com Chay', 'Vitamina Juices & Blends', 'Loving Hut', 'Vegefarm', "Brother Baby's BBQ", 'The Happy Hooligans', 'Queen Of Sheba', 'Raw Vittles', 'Clean Eating with Katie']


In [114]:
dist_vegan=veg_rest_sj4["response"]["groups"][0]["items"][8]["venue"]["location"]["distance"]
lat_vegan=veg_rest_sj4["response"]["groups"][0]["items"][8]["venue"]["location"]["lat"]
lng_vegan=veg_rest_sj4["response"]["groups"][0]["items"][8]["venue"]["location"]["lng"]
name_vegan=veg_rest_sj4["response"]["groups"][0]["items"][8]["venue"]["name"]

print(f"company name: {name_vegan}, company coordinates: {lat_vegan},{lng_vegan}, distance to the offices: {dist_vegan}")

company name: Vegefarm, company coordinates: 37.385706,-121.908328, distance to the offices: 2067


The nearest vegan restaurant is at 2km

In [289]:
# I make a query to the Foursquare API to look for basketball courts near this company.

api_function('37.3706126,-121.9219538', 'Basketball Court', 'basketball_court_sj4')


{'meta': {'code': 200, 'requestId': '5fb28dd6e6b2a16984a059b6'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'basketball court',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 37.420003872877636,
    'lng': -121.87698936462401},
   'sw': {'lat': 37.32647170912039, 'lng': -121.97387886047365}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a2f2f67f964a520b3981fe3',
       'name': 'Central YMCA',
       'location': {'address': '1717 The Alameda',
        'crossStreet': 'at Naglee Ave.',
        'lat': 37.33779578622328,
        'lng': -121.91989631992244,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [222]:
print(dist(basketball_court_sj4))
print("----------------------------------------------")
print(lat(basketball_court_sj4))
print("----------------------------------------------")
print(lng(basketball_court_sj4))
print("----------------------------------------------")
print(name(basketball_court_sj4))

[3657, 709, 908, 932, 986, 2890, 3170, 3662, 4115, 4229, 4237, 4801, 4441, 4713]
----------------------------------------------
[37.33779578622328, 37.37696075439453, 37.36441421508789, 37.37117385864258, 37.364402, 37.3881680172003, 37.377344606195855, 37.39918075740896, 37.37372, 37.33270914406301, 37.3753662109375, 37.40387815016547, 37.330723171109355, 37.3980712890625]
----------------------------------------------
[-121.91989631992244, -121.92266082763672, -121.91527557373047, -121.91143798828125, -121.913997, -121.94602505794903, -121.9567734963286, -121.94248645305073, -121.968306, -121.9253028797591, -121.96947479248047, -121.9565124356151, -121.92286373936807, -121.88139343261719]
----------------------------------------------
['Central YMCA', 'eBay Inc. Basketball Court', 'Brocade Courts', 'Basketball Gym', 'Skyport!', 'Montague Park', '3 Ring', 'mansion grove basketball court', "Rucker's Basketball Court", 'Hoover Gym', 'Podium Golden State Warriors Basketball Court', 'Lick

In [119]:
dist_basket=basketball_court_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_basket=basketball_court_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_basket=basketball_court_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_basket=basketball_court_sj4["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

dist_basket=basketball_court_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["distance"]
lat_basket=basketball_court_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["lat"]
lng_basket=basketball_court_sj4["response"]["groups"][0]["items"][2]["venue"]["location"]["lng"]
name_basket=basketball_court_sj4["response"]["groups"][0]["items"][2]["venue"]["name"]

print(f"company name: {name_basket}, company coordinates: {lat_basket},{lng_basket}, distance to the offices: {dist_basket}")

company name: eBay Inc. Basketball Court, company coordinates: 37.37696075439453,-121.92266082763672, distance to the offices: 709
company name: Brocade Courts, company coordinates: 37.36441421508789,-121.91527557373047, distance to the offices: 908


There are two basketball courts near the office

In [288]:
# I make a query to the Foursquare API to look for Starbucks near this company.

api_function('37.3706126,-121.9219538', 'starbucks', 'starbucks_sj4')


{'meta': {'code': 200, 'requestId': '5fb28db8aa0cc20f98e5d739'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'starbucks',
  'totalResults': 50,
  'suggestedBounds': {'ne': {'lat': 37.405360295285654,
    'lng': -121.8921408297042},
   'sw': {'lat': 37.3307940554626, 'lng': -121.94219457621179}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a6b4b07f964a5205cce1fe3',
       'name': 'Starbucks',
       'location': {'address': '1751 N 1st St',
        'lat': 37.37092547570862,
        'l

In [221]:
print(dist(starbucks_sj4))
print("----------------------------------------------")
print(lat(starbucks_sj4))
print("----------------------------------------------")
print(lng(starbucks_sj4))
print("----------------------------------------------")
print(name(starbucks_sj4))

[415, 120, 678, 1219, 207, 4059, 2227, 3093, 3106, 3374, 2812, 3835, 1726, 3070]
----------------------------------------------
[37.37092547570862, 37.371585845947266, 37.369529724121094, 37.36107375737311, 37.369186, 37.33418343, 37.35060173, 37.345295, 37.395157, 37.341431, 37.38324096, 37.40197092074824, 37.385382, 37.398163]
----------------------------------------------
[-121.91727116354467, -121.92254638671875, -121.92949676513672, -121.92873647557042, -121.92045, -121.9241052, -121.9221276, -121.936371, -121.90523999999999, -121.911642, -121.894416, -121.93991940591599, -121.927919, -121.923706]
----------------------------------------------
['Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks', 'Starbucks']


In [122]:
dist_starbucks=starbucks_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_starbucks=starbucks_sj4["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

dist_starbucks=starbucks_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["distance"]
lat_starbucks=starbucks_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["lat"]
lng_starbucks=starbucks_sj4["response"]["groups"][0]["items"][4]["venue"]["location"]["lng"]
name_starbucks=starbucks_sj4["response"]["groups"][0]["items"][4]["venue"]["name"]

print(f"company name: {name_starbucks}, company coordinates: {lat_starbucks},{lng_starbucks}, distance to the offices: {dist_starbucks}")

company name: Starbucks, company coordinates: 37.371585845947266,-121.92254638671875, distance to the offices: 120
company name: Starbucks, company coordinates: 37.369186,-121.92045, distance to the offices: 207


There are two Starbucks at 120 and 200 km from the office

In [287]:
# I make a query to the Foursquare API to look for pet hairdressers near this company.

api_function('37.3706126,-121.9219538', 'Pet Service', 'pet_service_sj4')


{'meta': {'code': 200, 'requestId': '5fb28da20cc1d6096f411fbb'},
 'response': {'suggestedRadius': 2288,
  'headerLocation': 'North San Jose',
  'headerFullLocation': 'North San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'query': 'pet service',
  'totalResults': 35,
  'suggestedBounds': {'ne': {'lat': 37.40474107281321,
    'lng': -121.88153774475427},
   'sw': {'lat': 37.32485747092281, 'lng': -121.98113218854394}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '49f6928cf964a520506c1fe3',
       'name': 'PetSmart',
       'location': {'address': '607 Coleman Ave',
        'crossStreet': 'at W Taylor St',
        'lat': 37.339942,
        'lng': -121.907791,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.339942,
          'lng'

In [220]:
print(dist(pet_service_sj1))
print("----------------------------------------------")
print(lat(pet_service_sj1))
print("----------------------------------------------")
print(lng(pet_service_sj1))
print("----------------------------------------------")
print(name(pet_service_sj1))

[3669, 2777, 4780, 3636, 5028, 2613, 2664, 2827, 2831, 2729, 3363, 3463, 3478, 3592]
----------------------------------------------
[37.339942, 37.3813788, 37.32848854373601, 37.3402263, 37.35306842721221, 37.36269, 37.36273680000001, 37.3796526024096, 37.379786, 37.34972481944317, 37.38021, 37.40111, 37.34043631759206, 37.343559452656756]
----------------------------------------------
[-121.907791, -121.8959331, -121.91154708173961, -121.9079148, -121.97660516837168, -121.95201, -121.9526368, -121.95375089364178, -121.953731, -121.90721511840819, -121.95995900000001, -121.91887, -121.91236495971678, -121.94683830386161]
----------------------------------------------
['PetSmart', 'Banfield Pet Hospital', 'Showtime Pet Grooming', 'Banfield Pet Hospital', 'Petco', 'Ohana Kennels', 'Bay Area K9 Association LLC.', 'A Pet Villa', 'A Pet Villa', 'DogWatch of the Bay Area & Northern California', 'Wag Hotels', 'Epic Dog Park', 'Primal Canine', "Juan's Groom Room"]


In [125]:
dist_dog=pet_service_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["distance"]
lat_dog=pet_service_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lat"]
lng_dog=pet_service_sj4["response"]["groups"][0]["items"][1]["venue"]["location"]["lng"]
name_dog=pet_service_sj4["response"]["groups"][0]["items"][1]["venue"]["name"]

print(f"company name: {name_dog}, company coordinates: {lat_dog},{lng_dog}, distance to the offices: {dist_dog}")

company name: Banfield Pet Hospital, company coordinates: 37.3813788,-121.8959331, distance to the offices: 2595


The nearest pet hairdresser is at 2.6km from the company